In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
image_files=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        image_files.extend([os.path.join(dirname,filename)])
print(len(image_files))

# Any results you write to the current directory are saved as output.

In [ ]:
print(image_files)

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import seaborn as sns
import cv2

import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [ ]:
from glob import glob
i_=0
#imagePatches = glob('../input/IDC_regular_ps50_idx5/**/*.png', recursive=True)
for filename in range(25):
    print(image_files[filename])
    image = cv2.imread(image_files[filename])
    im = cv2.resize(image, (50, 50)) 
    plt.subplot(5, 5, filename+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)); plt.axis('on')
    i_ += 1
    
# Plot Multiple Images


In [ ]:

def randomImages(a):
    r = random.sample(a, 4)
    plt.figure(figsize=(16,16))
    plt.subplot(131)
    plt.imshow(cv2.imread(r[0]))
    plt.subplot(132)
    plt.imshow(cv2.imread(r[1]))
    plt.subplot(133)
    plt.imshow(cv2.imread(r[2])); 
randomImages(image_files)

   


In [ ]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(image_files, patternZero)
classOne = fnmatch.filter(image_files, patternOne)
print("IDC(-)\n\n",classZero[0:10],'\n')
print("IDC(+)\n\n",classOne[0:10])

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('/kaggle/input/breast-histopathology-images/12935/0/12935_idx5_x451_y1751_class0.png',0)

hist,bins = np.histogram(img.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()
plt.subplot(121)
plt.plot(cdf_normalized, color = 'b')
plt.hist(img.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')

plt.show()
cdf_m = np.ma.masked_equal(cdf,0)
cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
cdf = np.ma.filled(cdf_m,0).astype('uint8')
img2 = cdf[img]
plt.subplot(122),plt.imshow(img2),plt.title('FHE')

In [ ]:
import cv2

img = cv2.imread('/kaggle/input/breast-histopathology-images/12935/0/12935_idx5_x451_y1751_class0.png',0)

# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(img)

cv2.imwrite('clahe_2.jpg',cl1)

In [ ]:
import cv2 as cv
import numpy as np

img = cv.imread('/kaggle/input/breast-histopathology-images/12935/0/12935_idx5_x451_y1751_class0.png',0) # loads in grayscale

alpha = 1
beta = 0
res = cv.multiply(img, alpha)
res = cv.add(res, beta)
res = cv.convertScaleAbs(img, alpha = alpha, beta = beta)
plt.subplot(122),plt.imshow(res),plt.title('FHE')
    #cv2.imwrite(image_files[filename], blur)
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
img = cv2.imread('/kaggle/input/breast-histopathology-images/12935/0/12935_idx5_x451_y1751_class0.png',0)
equ = cv2.equalizeHist(img)
res = np.hstack((img,equ)) #stacking images side-by-side
cv2.imwrite('res.png',res)


In [ ]:
hist,bins = np.histogram(res.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()
plt.subplot(121)
plt.plot(cdf_normalized, color = 'b')
plt.hist(img.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')


In [ ]:
import numpy as np
import cv2

img = cv2.imread('/kaggle/input/breast-histopathology-images/12935/0/12935_idx5_x451_y1751_class0.png',0)

# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(img)

cv2.imwrite('clahe_2.jpg',cl1)
plt.subplot(122),plt.imshow(cl1),plt.title('FHE')

In [ ]:
#apply Gaussian Filter
i=0
grayimg=[]
for filename in range(25):
    #print(image_files[filename])
    image = cv2.imread(image_files[filename])
    #img_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #blur = cv2.GaussianBlur( img_gray,(5,5),0)
    median = cv2.medianBlur(image,5)
    plt.subplot(121),plt.imshow(img_gray),plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img_gray),plt.title('FHE')
    #cv2.imwrite(image_files[filename], blur)
    plt.xticks([]), plt.yticks([])
    plt.show()

In [ ]:
# Segmentation using threshold
from pylab import *
from PIL import *

# create a new figure
figure()
gray()

for filename in range(25):
    
    original = cv2.imread(image_files[filename])
    grayimg = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(img_gray,5)
    
    ret, thresh = cv2.threshold(median , 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Displaying segmented images
  # show contours with origin upper left corner
    contour(thresh, origin='image')
    axis('equal')
    axis('off')


    figure()


    hist(thresh.flatten(), 128)

    show()
    


    figure()
 

In [ ]:
from os import listdir
base_path = '../input/breast-histopathology-images/IDC_regular_ps50_idx5/'
# We will save the base path to the patient folders, so that we can easily loop over them
patient_ids = listdir(base_path)

class_0_total = 0
class_1_total = 0
from pprint import pprint
for patient_id in patient_ids:
    class_0_files = listdir(base_path + patient_id + '/0')
    class_1_files = listdir(base_path + patient_id + '/1')

    class_0_total += len(class_0_files)
    class_1_total += len(class_1_files) 

total_images = class_0_total + class_1_total
    
print(f'Number of patches in Class 0: {class_0_total}')
print(f'Number of patches in Class 1: {class_1_total}')
print(f'Total number of patches: {total_images}')

In [ ]:


# note that we loop through the classes after looping through the 
# patient ids so that we avoid splitting our data into [all class 0 then all class 1]
# Segmentation using threshold
from pylab import *
from PIL import *
from tqdm.notebook import tqdm
tqdm().pandas();



# create a new figure
columns = ["patient_id",'x','y',"target","path"]
data_rows = []
i = 0
iss = 0
isss = 0
figure()
gray()

for filename in range(25):
    for patient_id in tqdm(patient_ids):
        for c in [0,1]:
            class_path = base_path + patient_id + '/' + str(c) + '/'
            imgs = listdir(class_path)
        
        # Extracting Image Paths
            img_paths = [class_path + img + '/' for img in imgs]
        
        # Extracting Image Coordinates
            img_coords = [img.split('_',4)[2:4] for img in imgs]
            x_coords = [int(coords[0][1:]) for coords in img_coords]
            y_coords = [int(coords[1][1:]) for coords in img_coords]

        for (path,x,y) in zip(img_paths,x_coords,y_coords):
            values = [patient_id,x,y,c,path]
            data_rows.append({k:v for (k,v) in zip(columns,values)})
# We create a new dataframe using the list of dicts that we generated above
        data = pd.DataFrame(data_rows)
        print(f'Shape of Dataframe: {data.shape}')
        data.head()
    
    original = cv2.imread(image_files[filename])
    grayimg = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(img_gray,5)
    
    ret, thresh = cv2.threshold(median , 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Displaying segmented images
  # show contours with origin upper left corner
    contour(thresh, origin='image')
    axis('equal')
    axis('off')


    figure()


    hist(thresh.flatten(), 128)

    show()
    


    figure()
   


In [ ]:
columns = ["patient_id",'x','y',"target","path"]
data_rows = []
i = 0
iss = 0
isss = 0

# note that we loop through the classes after looping through the 
# patient ids so that we avoid splitting our data into [all class 0 then all class 1]
for patient_id in tqdm(patient_ids):
    for c in [0,1]:
        class_path = base_path + patient_id + '/' + str(c) + '/'
        imgs = listdir(class_path)
        
        # Extracting Image Paths
        img_paths = [class_path + img + '/' for img in imgs]
        
        # Extracting Image Coordinates
        img_coords = [img.split('_',4)[2:4] for img in imgs]
        x_coords = [int(coords[0][1:]) for coords in img_coords]
        y_coords = [int(coords[1][1:]) for coords in img_coords]

        for (path,x,y) in zip(img_paths,x_coords,y_coords):
            values = [patient_id,x,y,c,path]
            data_rows.append({k:v for (k,v) in zip(columns,values)})
# We create a new dataframe using the list of dicts that we generated above
data = pd.DataFrame(data_rows)
print(f'Shape of Dataframe: {data.shape}')
data.head()

In [ ]:
# Return random sample indexes that are cancer positive and cancer negative of size 50
# replace = False means that no duplication is allowed
from pylab import *
from PIL import *
from skimage.feature import greycomatrix, greycoprops
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from skimage import io

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import plotly.express as px
figure()
gray()



#patient_ids = np.random.choice( data.patient_id.unique(), size=n_imgs, replace=False)
#fig,ax = plt.subplots(n_rows,n_cols,figsize = (30,30))


n_rows = 10
n_cols = 10

#Cancerous Patches
#fig,ax = plt.subplots(n_rows,n_cols,figsize = (30,30))
n_rows = 5
n_cols = 3
n_imgs = n_rows*n_cols
colors = ['pink', 'purple']
fig, ax = plt.subplots(n_rows,n_cols,figsize=(20, 27))

#fig, ax = plt.subplots(n_rows,n_cols,figsize=(20, 27))

patient_ids = np.random.choice( data.patient_id.unique(), size=n_imgs, replace=False)
positive_tissue = np.random.choice(data[data.target==1].index.values, size=100, replace=False)
negative_tissue = np.random.choice(data[data.target==0].index.values, size=100, replace=False)

for row in tqdm(range(n_rows)):
    for col in range(n_cols):
        # below is a counter to cycle through the image indexes
        idx = positive_tissue[col + n_cols*row]
        img = io.imread(data.loc[idx, "path"])
        patient_id = patient_ids[col + n_cols*row]
        patient_df = get_patient_df(patient_id)
        
        ax[row,col].scatter(patient_df.x.values, \
                            patient_df.y.values, \
                            c=patient_df.target.values,\
                            cmap=ListedColormap(colors), s=20)
        ax[row,col].set_title("patient " + patient_id)

       # ax[row,col].imshow(img[:,:,:])
       # ax[row,col].grid(False)
        
for row in tqdm(range(n_rows)):
    for col in range(n_cols):
        # below is a counter to cycle through the image indexes
        idx = positive_tissue[col + n_cols*row]
        img = io.imread(data.loc[idx, "path"])
        grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        median = cv2.medianBlur(grayimg,5)
        
        contour(median, origin='image')
        axis('equal')
        axis('off')
        figure()
        hist(median.flatten(), 128)
        show()
        figure()
        img_array = (median.flatten())

        img_array  = img_array.reshape(-1, 1).T



In [ ]:
def get_patient_df(patient_id):
    return data.loc[data['patient_id']== patient_id,:]


In [ ]:
print("Non Cancerous Patches")


#Non Cancerous Patches
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import plotly.express as px
n_cols = 3
n_imgs = n_rows*n_cols
colors = ['pink', 'purple']

fig, ax = plt.subplots(n_rows,n_cols,figsize=(20, 27))

patient_ids = np.random.choice( data.patient_id.unique(), size=n_imgs, replace=False)
#fig,ax = plt.subplots(n_rows,n_cols,figsize = (30,30))
for row in tqdm(range(n_rows)):
    for col in range(n_cols):
        # below is a counter to cycle through the image indices
        idx = negative_tissue[col + n_cols*row]
        img = io.imread(data.loc[idx, "path"])
        img = cv2.medianBlur(img,5)
        #ax[row,col].imshow(img[:,:,:])
        #ax[row,col].grid(False)
        patient_id = patient_ids[col + n_cols*row]
        patient_df = get_patient_df(patient_id)
        
        ax[row,col].scatter(patient_df.x.values, \
                            patient_df.y.values, \
                            c=patient_df.target.values,\
                            cmap=ListedColormap(colors), s=20)
        ax[row,col].set_title("patient " + patient_id)

        
for row in tqdm(range(n_rows)):
    for col in range(n_cols):
        # below is a counter to cycle through the image indexes
        idx = positive_tissue[col + n_cols*row]
        grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        median = cv2.medianBlur(grayimg,5)
        img = io.imread(data.loc[idx, "path"])
        contour(median, origin='image')
        patient_id = patient_ids[col + n_cols*row]
        patient_df = get_patient_df(patient_id)
        ax[row,col].set_title("patient " + patient_id)
        axis('equal')
        axis('on')
        figure()
        hist(median.flatten(), 128)
        show()
        figure()
        img_array = (median.flatten())

        img_array  = img_array.reshape(-1, 1).T


In [ ]:
from sklearn.model_selection import train_test_split

def split_data(data,train_test_percentage = 0.7,percentage_of_dataset = 0.2):
    # get number of datapoints to split from the total dataset
    n_points = round(data.shape[0]*0.2)
    # retrieve the desired initial split of the data based on the specified percentage of the dataset
    data = data[0:n_points]
    # retrieve all the columns except the target columns and store them in X
    X = data.loc[:,data.columns != 'target']
    # retrieve the target column and store them in y 
    y = data.loc[:,['target']]
    # Split the model to training and testing sets for both the features and the targets
    X_train, X_test, y_train, y_test =  train_test_split(X, y, train_size=train_test_percentage, shuffle = False)
    return [X_train, X_test, y_train, y_test]


In [ ]:
X_train, X_test, y_train, y_test = split_data(data)


In [ ]:
from skimage.color import rgb2gray
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
from sklearn.cluster import KMeans

for i in range(10):
    print(i)
    img_path = X_train.path[i]
# reading the images
    rgb_img = io.imread(img_path)
    gray = io.imread(img_path, as_gray = True)
    kmeans = KMeans(n_clusters=5, random_state=0).fit(gray)
    pic2show = kmeans.cluster_centers_[kmeans.labels_]
    plt.subplot(121),plt.imshow(pic2show),plt.title('After KMeans')
    plt.xticks([]), plt.yticks([])
    plt.show()
   # converting to grayscale
    #gray = rgb2gray(image)

# defining the sobel filters
    
   
    


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(pic_n)
pic2show = kmeans.cluster_centers_[kmeans.labels_]

In [ ]:
import os
import gc
import cv2
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# ablumentations for easy image augmentation for input as well as output
import albumentations as albu
# from albumentations import torch as AT
plt.style.use('bmh')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
from skimage.color import rgb2hsv
import os
img = Image.open(r'/kaggle/input/breast-histopathology-images/9176/0/9176_idx5_x1201_y801_class0.png')
x = np.array(img)
imshow(x)

In [ ]:
z = np.dstack((x,rgb2hsv(x)))
z.shape
vectorized = np.float32(z.reshape((-1,6)))
vectorized.shape
kmeans = KMeans(random_state=0, init='random', n_clusters=8)
labels = kmeans.fit_predict(vectorized)

In [ ]:


labels.shape
pic = labels.reshape(50,50)
f, axarr = plt.subplots(1,2,figsize=(15,15))
axarr[0].set_xlabel('Original Image', fontsize=12)
axarr[1].set_xlabel('Segmented Image', fontsize=12)  
axarr[0].imshow(x)
axarr[1].imshow(pic)


In [ ]:
for filename in range(25):
    img = Image.open(image_files[filename])
    x = np.array(img)
    imshow(x)
    x = np.array(img)
    z = np.dstack((x,rgb2hsv(x)))
    z.shape
    vectorized = np.float32(z.reshape((-1,6)))
    kmeans = KMeans(random_state=32, init='random', n_clusters=6)
    james = kmeans.fit_predict(vectorized)
    pic = james.reshape(50,50)
    f, axarr = plt.subplots(1,2,figsize=(15,15))
    axarr[0].set_xlabel('Original Image', fontsize=12)
    axarr[1].set_xlabel('Segmented Image', fontsize=12)  
    axarr[0].imshow(x)
    axarr[1].imshow(pic)

In [ ]:
#Watershed
for filename in range(25):
    imgs=cv2.imread('/kaggle/input/breast-histopathology-images/9176/1/9176_idx5_x2201_y1201_class1.png')
    img = cv2.imread('/kaggle/input/breast-histopathology-images/9176/1/9176_idx5_x2201_y1201_class1.png')
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # noise removal
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2) 
    # sure background area
    sure_bg = cv2.dilate(closing,kernel,iterations=3)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(closing,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]
    #plt.imshow(markers)
    #plt.imshow(img)
    f, axarr = plt.subplots(1,2,figsize=(15,15))
    axarr[0].set_xlabel('Original Image', fontsize=12)
    axarr[1].set_xlabel('Segmented Image', fontsize=12)  
    axarr[0].imshow(imgs)
    axarr[1].imshow(img)

In [ ]:
img=cv2.imread('/kaggle/input/breast-histopathology-images/14304/0/14304_idx5_x1151_y2201_class0.png')
#img = cv2.imread('/kaggle/input/breast-histopathology-images/9176/1/9176_idx5_x2201_y1201_class1.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # noise removal
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2) 
    # sure background area
sure_bg = cv2.dilate(closing,kernel,iterations=3)
    # Finding sure foreground area
dist_transform = cv2.distanceTransform(closing,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
markers = markers+1
    # Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
#plt.imshow(markers)
plt.imshow(img)

In [ ]:
import cv2
import matplotlib.pyplot as plt


def compute_simple_mask(image):
  image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return cv2.threshold(image_grayscale, 127, 255, 0)[1]


image_path = '/kaggle/input/breast-histopathology-images/9176/1/9176_idx5_x2201_y1201_class1.png'  # e.g. https://notebooks.azure.com/clewolff/libraries/otsu/raw/golf1.jpg
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
mask_simple = compute_simple_mask(image)


def show_mask(mask, image, title='', mask_color=(255, 0, 0)):
    display_image = image.copy()
    display_image[mask != 0] = mask_color
    plt.imshow(display_image)
    plt.title(title)
    plt.axis('off')
    plt.show()


show_mask(mask_simple, image, title='Simple grayscale thresholding')

In [ ]:
def show_intensity_histogram(image):
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    plt.hist(image_grayscale.ravel(), 256)
    plt.title('Intensity histogram')
    plt.ylabel('Number of pixels')
    plt.xlabel('Light intensity')
    plt.show()


show_intensity_histogram(image)

In [ ]:
def compute_otsu_mask(image):
  image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return cv2.threshold(image_grayscale, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
image_path = '/kaggle/input/breast-histopathology-images/9176/1/9176_idx5_x2201_y1201_class1.png'  # e.g. https://notebooks.azure.com/clewolff/libraries/otsu/raw/golf1.jpg
image = cv2.imread(image_path)
mask_otsu = compute_otsu_mask(image)
show_mask(mask_otsu, image, title='Otsu grayscale thresholding')

In [ ]:
import numpy as np

kernel = np.ones((5, 5), np.uint8)

mask_otsu_clean = cv2.morphologyEx(mask_otsu, cv2.MORPH_OPEN, kernel, iterations=2)
mask_otsu_clean = cv2.erode(mask_otsu_clean, kernel, iterations=2)
mask_otsu_clean = cv2.dilate(mask_otsu_clean, kernel, iterations=5)

show_mask(mask_otsu_clean, image, title='Otsu grayscale thresholding with morphological cleanup')

In [ ]:
def compute_otsu_mask_hue(image):
    image_hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)

    hue, lightness, saturation = np.split(image_hls, 3, axis=2)

    hue = hue.reshape((hue.shape[0], hue.shape[1]))

    otsu = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    otsu_mask = otsu != 255

    return otsu_mask


mask_otsu_hue = compute_otsu_mask_hue(image)
show_mask(mask_otsu_hue, image, title='Otsu thresholding on the hue channel')

In [ ]:
def compute_otsu_mask_shadows(image, shadow_percentile=5):
    image_hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)

    hue, lightness, saturation = np.split(image_hls, 3, axis=2)
    hue = hue.reshape((hue.shape[0], hue.shape[1]))

    otsu = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    otsu_mask = otsu != 255

    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = np.split(image_lab, 3, axis=2)
    l = l.reshape((l.shape[0], l.shape[1]))

    shadow_threshold = np.percentile(l.ravel(), q=shadow_percentile)
    shadows_mask = l < shadow_threshold

    mask = otsu_mask ^ shadows_mask

    return mask


image_path2 = '/kaggle/input/breast-histopathology-images/14304/0/14304_idx5_x1151_y2201_class0.png'  # e.g. https://notebooks.azure.com/clewolff/libraries/otsu/raw/golf2.jpg
image2 = cv2.imread(image_path2)
mask_otsu_shadows = compute_otsu_mask_shadows(image2)
show_mask(mask_otsu_shadows, image2, title='Otsu thresholding on the hue channel with shadow removal')

In [ ]:
n_classes=2
out = Conv2D(n_classes ,(1, 1) , padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model

model = get_segmentation_model(img_input ,  out ) # this would build the segmentation model

In [ ]:
model.train( 
    train_images =  "/kaggle/input/breast-histopathology-images/9176/0/9176_idx5_x2101_y151_class0.png",
    train_annotations = "/kaggle/input/breast-histopathology-images/9176/0",
    checkpoints_path = "checkpoints/vgg_unet_1" , epochs=5
)

In [ ]:
pip install pillow

In [ ]:
import random
import matplotlib.pylab as plt
import seaborn as sns
import cv2
#from scipy.misc import imresize, imread
from PIL import Image
import numpy as np
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [ ]:
from os import listdir
listdir('/kaggle/input')[:10]
def proc_images():
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """ 
    x = []
    y = []
    WIDTH = 50
    HEIGHT = 50
    for filename in range(len(image_files)):
        print(image_files[filename])
        img=image_files[filename]
        full_size_image = cv2.imread(image_files[filename])
        grayimg = cv2.cvtColor(full_size_image, cv2.COLOR_BGR2GRAY)
        median = cv2.medianBlur(grayimg,5)
        x.append(cv2.resize(median, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        
        if img in classZero:
            
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
    return x,y
X,Y = proc_images()
X1 = np.array(X)
df = pd.DataFrame()
df["images"]=X
df["labels"]=Y
len(image_files)

X2=df["images"]
Y2=df["labels"]

type(X2)

X2=np.array(X2)

X2.shape

imgs0=[]
imgs1=[]
imgs0 = X2[Y2==0] # (0 = no IDC, 1 = IDC)
imgs1 = X2[Y2==1] 






In [ ]:
def describeData(a,b):
    print('Total number of images: {}'.format(len(a)))
    print('Number of IDC(-) Images: {}'.format(np.sum(b==0)))
    print('Number of IDC(+) Images: {}'.format(np.sum(b==1)))
    print('Percentage of positive images: {:.2f}%'.format(100*np.mean(b)))
    print('Image shape (Width, Height, Channels): {}'.format(a[0].shape))

In [ ]:
def get_classes_split(series):
    ratio = np.round(series.value_counts()/series.count()*100,decimals = 1)
    return ratio

In [ ]:
(get_classes_split(data['target']))

In [ ]:
from sklearn.model_selection import train_test_split
# Set the patient ids as indices for the dataframe
shuffled_data = data.set_index('patient_id')
# Select all columns except the target column and store it in X
X = shuffled_data.loc[:, shuffled_data.columns != 'target']
# Select the target column and store it in y 
y = shuffled_data['target']

# OS stands for 'Out of Sample'
X_data, OSX_df, y_data, OSy_df = train_test_split(X, y, test_size=0.1, shuffle = False)
# We split it even further and obtain the training and testing dataframes
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_data, y_data, test_size=0.3, shuffle = False)

#Let's ensure that our classes split is maintained after the shuffling and splitting.

print(f'y_data: {get_classes_split(y_data)}')
display(get_classes_split(y_test_df))
display(get_classes_split(OSy_df))



In [ ]:
X_train_df

In [ ]:
def visualise_breast_tissue(patient_id, df = data,pred = False, crop_dimension = [50,50]):
    # Plotting Settings
    plt.xticks([])
    plt.yticks([])
    # Get patient dataframe
    p_df = get_patient_df(patient_id)
    # Get the dimensions of the breast tissue image
    max_coord = np.max((*p_df.x,*p_df.y))
    # Allocate an array to fill image pixels in,use uint8 type as you don't need an int over 255
    grid = 255*np.ones(shape = (max_coord + crop_dimension[0], max_coord + crop_dimension[1], 3)).astype(np.uint8)
    mask = 255*np.ones(shape = (max_coord + crop_dimension[0], max_coord + crop_dimension[1], 3)).astype(np.uint8)
    # Replace array values with values of the image
    for x,y,target,path in zip(p_df['x'],p_df['y'],p_df['target'],p_df['path']):
        try:
            img = io.imread(path)
            # Replace array values with cropped image values
            grid[y:y+crop_dimension[1],x:x+crop_dimension[0]] = img
            # Check if target is cancerous or not
            if target != 0:
                # If the target is cancerous then, replace array values with the color blue
                mask[y:y+crop_dimension[1],x:x+crop_dimension[0]] = [0,0,255]
        except: pass
    # if prediction is not specifies then show the image normally
    if pred == False:
        io.imshow(grid)
        img = grid
    # if prediction is specified then apply a mask to the areas that contain predicted cancerous cells
    else:
        # Specify the desired alpha value
        alpha = 0.78
        # This is step is very important, adding 2 numpy arrays sets the values to float64, which is why convert them back to uint8
        img = (mask * (1.0 - alpha) + grid * alpha).astype('uint8')
        io.imshow(img)
    return img

In [ ]:
fig, ax = plt.subplots(n_rows,n_cols,figsize=(20, 27))

for row in range(n_rows):
    for col in range(n_cols):
        p_id = patient_ids[col + n_cols*row]
        
        img = visualise_breast_tissue(p_id, pred = True)
        ax[row,col].grid(False)
        ax[row,col].set_xticks([])
        ax[row,col].set_yticks([])
        ax[row,col].set_title("Breast tissue slice of patient: " + p_id)        
        ax[row,col].imshow(img)

In [ ]:
# Dataframe containing cancerous images
c_df = data.loc[data.target == 1,:]
# Dataframe containing normal images
n_df = data.loc[data.target == 0,:]

fraction_c = np.round(0.7*c_df.shape[0]).astype(int)
fraction_n = np.round(0.2*n_df.shape[0]).astype(int)

rest_c_df = c_df.iloc[fraction_c:-1]
rest_n_df = n_df.iloc[fraction_n:-1]

c_df = c_df.iloc[0: fraction_c]
n_df = n_df.iloc[0: fraction_n]

nc = c_df.shape[0]
nn = n_df.shape[0]

nrc = rest_c_df.shape[0]
nrn = rest_n_df.shape[0]

total_test = nn+nc
total_train = nrn + nrc

print("Testing Data:") 
print(f'percent cancerous : {round(nc/total_test*100,1)}%')
print(f'percent non-cancerous : {round(nn/total_test*100,1)}%\n')
print("Training Data:")
print(f'percent cancerous : {round(nrc/total_train*100,1)}%')
print(f'percent non-cancerous : {round(nrn/total_train*100,1)}%')

In [ ]:
n_train_df = c_df.append(n_df, sort = True).reset_index(drop=True)
n_test_df = rest_c_df.append(rest_n_df,sort = True).reset_index(drop=True)

In [ ]:
n_train_df

In [ ]:
import gc
from sklearn.decomposition import IncrementalPCA
def rgb_to_grayscale(img_paths, batch_size = 15000):
    # get the total number of images
    num_of_imgs = img_paths.shape[0]
    # initialize counter that keeps track of position of image being loaded
    pos = 0
    # initialize empty array in order fill in the image values
    grid = np.zeros((num_of_imgs*2500, 3))

    for img_path in tqdm(img_paths, total=num_of_imgs):
        # Read the image into a numpy array 
        img = io.imread(img_path)
        # reshape the image to such that the rgb values are the columns of the matrix
        img = img.reshape(-1, 3)
        # replace the empty array with the values inside the image
        grid[pos: pos + img.shape[0],:] = img
        # update position counter
        pos += img.shape[0]
        
    # initialize pca to reduce rgb scale to a single dimensional scale
    ipca = IncrementalPCA(n_components=1, batch_size=batch_size)
    # fit pca object to the contents within the grid
    ipca.fit(grid)
    # delete grid to free up sum memory
    del grid
    gc.collect()
    
    return ipca

In [ ]:
img_paths = n_train_df['path']
rgb_pca = rgb_to_grayscale(img_paths)

In [ ]:
rgb_pca

In [ ]:
#NEW CODING WITH CNN

In [ ]:
imagePatches = image_files

In [ ]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)
print("IDC(-)\n\n",classZero[0:5],'\n')
print("IDC(+)\n\n",classOne[0:5])

In [ ]:
def proc_images(lowerIndex,upperIndex):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """ 
    x = []
    y = []
    WIDTH = 50
    HEIGHT = 50
    for img in imagePatches[lowerIndex:upperIndex]:
        full_size_image = cv2.imread(img)
        #grayimg = cv2.cvtColor(full_size_image, cv2.COLOR_BGR2GRAY)
        median = cv2.medianBlur(full_size_image,5)
        x.append(cv2.resize(median, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        if img in classZero:
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
    return x,y

In [ ]:
X,Y = proc_images(0,5000)

In [ ]:
X1 = np.array(X)

X1.shape



In [ ]:
df = pd.DataFrame()
df["images"]=X
df["labels"]=Y

X2=df["images"]
Y2=df["labels"]



In [ ]:
type(X2)



X2=np.array(X2)

X2.shape



imgs0=[]
imgs1=[]
imgs0 = X2[Y2==0] # (0 = no IDC, 1 = IDC)
imgs1 = X2[Y2==1] 

def describeData(a,b):
    print('Total number of images: {}'.format(len(a)))
    print('Number of IDC(-) Images: {}'.format(np.sum(b==0)))
    print('Number of IDC(+) Images: {}'.format(np.sum(b==1)))
    print('Percentage of positive images: {:.2f}%'.format(100*np.mean(b)))
    print('Image shape (Width, Height, Channels): {}'.format(a[0].shape))
describeData(X2,Y2)



In [ ]:
def plotTwo(a,b): 
    """
    Plot a bunch of numpy arrays sorted by label
    """
    for row in range(3):
        plt.figure(figsize=(20, 10))
        for col in range(3):
            plt.subplot(1,8,col+1)
            plt.title('IDC (-)')
            plt.imshow(a[0+row+col])
            plt.axis('off')       
            plt.subplot(1,8,col+4)
            plt.title('IDC (+)')
            plt.imshow(b[0+row+col])
            plt.axis('off')
plotTwo(imgs0, imgs1) 

In [ ]:
def plotHistogram(a):
    """
    Plot histogram of RGB Pixel Intensities
    """
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(a)
    plt.axis('off')
    plt.title('IDC(+)' if Y[1] else 'IDC(-)')
    histo = plt.subplot(1,2,2)
    histo.set_ylabel('Count')
    histo.set_xlabel('Pixel Intensity')
    n_bins = 30
    plt.hist(a[:,:,0].flatten(), bins= n_bins, lw = 0, color='r', alpha=0.5);
    plt.hist(a[:,:,1].flatten(), bins= n_bins, lw = 0, color='g', alpha=0.5);
    plt.hist(a[:,:,2].flatten(), bins= n_bins, lw = 0, color='b', alpha=0.5);
plotHistogram(X2[100])

In [ ]:
X=np.array(X)
X=X/255.0

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape
X_test.shape

In [ ]:
dict_characters = {0: 'IDC(-)', 1: 'IDC(+)'}
print(df.head(10))
print("")
print(dict_characters)

In [ ]:
dist = df['labels'].value_counts()
lab = df['labels']

dist



sns.countplot(df['labels'])



In [ ]:
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(Y_train, num_classes = 2)
y_testHot = to_categorical(Y_test, num_classes = 2)

In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
print("Old Class Weights: ",class_weight)
from sklearn.utils import class_weight
class_weight2 = class_weight.compute_class_weight('balanced', np.unique(Y_trainRos), Y_trainRos)
print("New Class Weights: ",class_weight2)

In [ ]:
#comsume more memory don't run
# Deal with imbalanced class sizes below
# Make Data 1D for compatability upsampling methods
X_trainShape = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_testShape = X_test.shape[1]*X_test.shape[2]*X_test.shape[3]
X_trainFlat = X_train.reshape(X_train.shape[0], X_trainShape)
X_testFlat = X_test.reshape(X_test.shape[0], X_testShape)
#print("X_train Shape: ",X_train.shape)
#print("X_test Shape: ",X_test.shape)
#print("X_trainFlat Shape: ",X_trainFlat.shape)
#print("X_testFlat Shape: ",X_testFlat.shape)

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#ros = RandomOverSampler(ratio='auto')
ros = RandomUnderSampler(sampling_strategy='auto')
X_trainRos, Y_trainRos = ros.fit_sample(X_trainFlat, Y_train)
X_testRos, Y_testRos = ros.fit_sample(X_testFlat, Y_test)

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_trainRosHot = to_categorical(Y_trainRos, num_classes = 2)
Y_testRosHot = to_categorical(Y_testRos, num_classes = 2)
#print("X_train: ", X_train.shape)
#print("X_trainFlat: ", X_trainFlat.shape)
#print("X_trainRos Shape: ",X_trainRos.shape)
#print("X_testRos Shape: ",X_testRos.shape)
#print("Y_trainRosHot Shape: ",Y_trainRosHot.shape)
#print("Y_testRosHot Shape: ",Y_testRosHot.shape)

for i in range(len(X_trainRos)):
    height, width, channels = 50,50,3
    X_trainRosReshaped = X_trainRos.reshape(len(X_trainRos),height,width,channels)
#print("X_trainRos Shape: ",X_trainRos.shape)
#print("X_trainRosReshaped Shape: ",X_trainRosReshaped.shape)

for i in range(len(X_testRos)):
    height, width, channels = 50,50,3
    X_testRosReshaped = X_testRos.reshape(len(X_testRos),height,width,channels)
#print("X_testRos Shape: ",X_testRos.shape)
#print("X_testRosReshaped Shape: ",X_testRosReshaped.shape)

dfRos = pd.DataFrame()
dfRos["labels"]=Y_trainRos
labRos = dfRos['labels']
distRos = lab.value_counts()
sns.countplot(labRos)
print(dict_characters)

In [ ]:
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy',allow_pickle=True)[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    total=sum(sum(cm))
    accuracy=(cm[0,0]+cm[1,1])/total
    specificity=cm[1,1]/(cm[1,0]+cm[1,1])
    sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
    print(cm)
    print(f'Accuracy: {accuracy}')
    print(f'Specificity: {specificity}')
    print(f'Sensitivity: {sensitivity}')
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    #plt.clf()
    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

batch_size = 128
num_classes = 2
epochs = 8
img_rows,img_cols=50,50
input_shape = (img_rows, img_cols, 3)
e = 2

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,strides=e))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
cnn=model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

a = X_train
b = y_trainHot
c = X_test
d = y_testHot
epochs = 10

history = model.fit_generator(datagen.flow(a,b, batch_size=32),
                        steps_per_epoch=len(a)/2, 
                              epochs=epochs,validation_data = [c, d],
                              callbacks = [MetricsCheckpoint('logs')])

y_pred = model.predict(c)

Y_pred_classes = np.argmax(y_pred,axis=1) 
Y_true = np.argmax(d,axis=1)

dict_characters = {0: 'IDC(-)', 1: 'IDC(+)'}

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = list(dict_characters.values())) 
plt.show()



In [ ]:
model.summary()

In [ ]:
def runKerasCNNAugment(a,b,c,d,e,f):
    """
    Run Keras CNN: https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py
    """
    batch_size = 128
    num_classes = 2
    epochs = 8
#     img_rows, img_cols = a.shape[1],a.shape[2]
    img_rows,img_cols=50,50
    input_shape = (img_rows, img_cols, 3)
    model2 = Sequential()
    model2.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape,strides=e))
    model2.add(Conv2D(64, (3, 3), activation='relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Dropout(0.25))
    model2.add(Flatten())
    model2.add(Dense(128, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(num_classes, activation='softmax'))
    model2.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
    history = model2.fit_generator(datagen.flow(a,b, batch_size=32),
                        steps_per_epoch=len(a) / 32, epochs=epochs,class_weight=f, validation_data = [c, d],callbacks = [MetricsCheckpoint('logs')])
    score = model2.evaluate(c,d, verbose=0)
    print('\nKeras CNN #1C - accuracy:', score[1],'\n')
    y_pred = model2.predict(c)
    map_characters = {0: 'IDC(-)', 1: 'IDC(+)'}
    print('\n', sklearn.metrics.classification_report(np.where(d > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='')    
    Y_pred_classes = np.argmax(y_pred,axis=1) 
    Y_true = np.argmax(d,axis=1) 
    plotKerasLearningCurve()
    plt.show()  
    plot_learning_curve(history)
    plt.show()
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    plot_confusion_matrix(confusion_mtx, classes = list(dict_characters.values())) 
    plt.show()
runKerasCNNAugment(X_trainRosReshaped, Y_trainRosHot, X_testRosReshaped, Y_testRosHot,2,class_weight)

In [ ]:
# concatenate two models, doesn't three
concat_a = tf.keras.layers.concatenate([model.output, 
                                      model2.output])
model_a = tf.keras.Model([model.input, model2.input], concat_a)

concat = tf.keras.layers.concatenate([model_a.output, 
                                        model3.output])

dense = tf.keras.layers.Dense(1024)(concat)
relu = tf.keras.layers.LeakyReLU(alpha=0.3)(dense)
normalize = tf.keras.layers.BatchNormalization()(relu)
out = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(normalize)

# nested list
model = tf.keras.Model([[model_1.input, model_2.input], model_3.input], out)

model.summary()

optimizer = RMSprop()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# simple list
history = model.fit([trainX, trainX, trainX], trainY)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model1 = Sequential()
#model1.add(Dropout(0.25))
model1.add(Flatten())
mlp=model1.add(Dense(128, activation='relu', input_shape=input_shape))
#model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))
model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history1 = model1.fit(a, b, batch_size=32, epochs=10, verbose=1,validation_data=(c,d))
model1.summary()

                            



In [ ]:
#MLP
test_scores = model1.evaluate(c,d, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [ ]:
from keras.models import Model
from keras.layers import concatenate
from keras.layers import PReLU

merged_layers = concatenate([model.output, model1.output])
x = BatchNormalization()(merged_layers)
x = Dense(128)(x)
x = PReLU()(x)
x = Dropout(0.2)(x)
x = Dense(1)(x)
x = BatchNormalization()(x)

out1 = Activation('sigmoid')(x)
out2 = Activation('sigmoid')(x)
merged_model = Model([model.input, model1.input], [out1,out2])

merged_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
#merged_model.metrics.accuracy(c,d)
#history2 = merged_model.fit(a, b, batch_size=32, epochs=10, verbose=1,validation_data=(c,d))
#merged_model.fit(model,out1)


In [ ]:
merged_model.summary()

In [ ]:
X = [np.zeros((1,50,50,1))] * 2
y = np.ones((1,1))
merged_model.fit(X, y)

In [ ]:
#working good

In [ ]:
from keras.layers import Dropout
input_shape = (img_rows, img_cols, 3)

model_reg = Sequential()
model_reg.add(Dense(512, activation='relu', input_shape=input_shape))

model_reg.add(Flatten())
model_reg.add(Dropout(0.5))

model_reg.add(Dense(num_classes, activation='softmax'))
model_reg.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history_reg = model_reg.fit(a,b, batch_size=32, epochs=20, verbose=1,validation_data=(c,d))

#Plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history_reg.history['loss'],'r',linewidth=3.0)
plt.plot(history_reg.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history_reg.history['accuracy'],'r',linewidth=3.0)
plt.plot(history_reg.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
model_reg.predict_classes(c[[0],:])
model_reg.predict(c[[0],:])

In [ ]:
#more dense CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

seq_length = 64

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#model.fit(a,b, batch_size=16, epochs=10)
#score = model.evaluate(c,d, batch_size=16)
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

a = X_train
b = y_trainHot
c = X_test
d = y_testHot
epochs = 10

history = model.fit_generator(datagen.flow(a,b, batch_size=32),
                        steps_per_epoch=len(a)/2, 
                              epochs=epochs,validation_data = [c, d],
                              callbacks = [MetricsCheckpoint('logs')])


In [ ]:
a = X_train
b = y_trainHot
c = X_test
d = y_testHot



In [ ]:
a = X_train
b = y_trainHot
c = X_test
d = y_testHot
epochs = 10


y_pred = model1.predict(c)

Y_pred_classes = np.argmax(y_pred,axis=1) 
Y_true = np.argmax(d,axis=1)

dict_characters = {0: 'IDC(-)', 1: 'IDC(+)'}

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = list(dict_characters.values())) 
plt.show()
test_scores = model1.evaluate(c,d, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

model3 = keras.Sequential([
    keras.layers.Flatten(input_shape=input_shape),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])


In [ ]:
model3 = keras.Sequential([ keras.layers.Flatten(input_shape=input_shape), keras.layers.Dense(128, activation='relu'), keras.layers.Dense(10) ])

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model3.fit(a, b, batch_size=32, epochs=10, verbose=1,validation_data=(c,d))



In [ ]:



#merged_model.fit(x,y,epochs=20,batch_size=10)
merged_model.fit(a,b, epochs=10)


In [ ]:
from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import scale
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import average
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, SpatialDropout1D
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import plot_model


D = 0.2
S = 1
img_rows,img_cols=50,50
input_shape = (img_rows, img_cols, 3)

seed(S)
### INPUT DATA
inputs = input_shape
### DEFINE A MULTILAYER PERCEPTRON NETWORK

mlp_net = Dense(32, activation = 'relu', kernel_initializer = 'he_uniform')(inputs)
mlp_net = Dropout(rate = D, seed = S)(mlp_net)
model.add(Flatten())
mlp_net = Dense(32, activation = 'relu', kernel_initializer = 'he_uniform')(mlp_net)
mlp_net = Dropout(rate = D, seed = S)(mlp_net)
model.add(Flatten())
mlp_out = Dense(num_classes, activation = 'sigmoid')(mlp_net)


mlp_mdl = Model(inputs = inputs, outputs = mlp_out)


In [ ]:
### DEFINE A CONVOLUTIONAL NETWORK 
cnv_net = Reshape((a.shape[1], 1))(inputs)
cnv_net = Conv1D(128, 4, activation = 'relu', padding = "same", kernel_initializer = 'he_uniform')(cnv_net)
cnv_net = MaxPooling1D(2)(cnv_net)
cnv_net = SpatialDropout1D(D)(cnv_net)
cnv_net = Flatten()(cnv_net)
cnv_out = Dense(1, activation = 'sigmoid')(cnv_net)
cnv_mdl = Model(inputs = inputs, outputs = cnv_out)
### COMBINE MLP AND CNV
con_out = concatenate()([mlp_out, cnv_out])
con_mdl = Model(inputs = inputs, outputs = con_out)
sgd = SGD(lr = 0.1, momentum = 0.9)
con_mdl.compile(optimizer = sgd, loss = 'binary_crossentropy',  metrics = ['binary_accuracy'])
con_mdl.fit(a, b, batch_size = 2000, epochs = 50, verbose = 0)
plot_model(con_mdl, to_file = 'model.png', show_shapes = True, show_layer_names = True)### DEFINE A CONVOLUTIONAL NETWORK 
cnv_net = Reshape((a.shape[1], 1))(inputs)
cnv_net = Conv1D(128, 4, activation = 'relu', padding = "same", kernel_initializer = 'he_uniform')(cnv_net)
cnv_net = MaxPooling1D(2)(cnv_net)
cnv_net = SpatialDropout1D(D)(cnv_net)
cnv_net = Flatten()(cnv_net)
cnv_out = Dense(1, activation = 'sigmoid')(cnv_net)
cnv_mdl = Model(inputs = inputs, outputs = cnv_out)
### COMBINE MLP AND CNV
con_out = concatenate()([mlp_out, cnv_out])
con_mdl = Model(inputs = inputs, outputs = con_out)
sgd = SGD(lr = 0.1, momentum = 0.9)
con_mdl.compile(optimizer = sgd, loss = 'binary_crossentropy',  metrics = ['binary_accuracy'])
con_mdl.fit(a, b, batch_size = 2000, epochs = 50, verbose = 0)
plot_model(con_mdl, to_file = 'model.png', show_shapes = True, show_layer_names = True)### DEFINE A CONVOLUTIONAL NETWORK 
cnv_net = Reshape((a.shape[1], 1))(inputs)
cnv_net = Conv1D(128, 4, activation = 'relu', padding = "same", kernel_initializer = 'he_uniform')(cnv_net)
cnv_net = MaxPooling1D(2)(cnv_net)
cnv_net = SpatialDropout1D(D)(cnv_net)
cnv_net = Flatten()(cnv_net)
cnv_out = Dense(1, activation = 'sigmoid')(cnv_net)
cnv_mdl = Model(inputs = inputs, outputs = cnv_out)
### COMBINE MLP AND CNV
con_out = concatenate()([mlp_out, cnv_out])
con_mdl = Model(inputs = inputs, outputs = con_out)
sgd = SGD(lr = 0.1, momentum = 0.9)
con_mdl.compile(optimizer = sgd, loss = 'binary_crossentropy',  metrics = ['binary_accuracy'])
con_mdl.fit(a, b, batch_size = 2000, epochs = 50, verbose = 0)
plot_model(con_mdl, to_file = 'model.png', show_shapes = True, show_layer_names = True)

In [ ]:
def all_cnn(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return modelall_cnn_model = all_cnn(model_input)
_, all_cnn_weight_file = compile_and_train(all_cnn_model, NUM_EPOCHS)

In [ ]:
combinedInput = concatenate([history, history1])
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [ ]:
plotKerasLearningCurve()
plt.show()  

In [ ]:
plot_learning_curve(history)
plt.show()

In [ ]:
clf = AdaBoostClassifier(model, n_estimators=3, algorithm="SAMME.R", learning_rate=0.5)